# I. Tweet Data Cleaning



In [11]:
## Tweet Table

In [12]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json
import time

In [13]:
#function to cheack whether 
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [14]:
#making mongoDB database and collection
client = MongoClient('localhost', 27017)
db = client['mydatabase']
db.tweet.drop()
tweet = db['tweet']

In [15]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [16]:
#yelin directory '/Users/yelin/Downloads/corona-out-2/corona-out-2'
#Mo 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-2'
file = 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-2'


In [17]:
%%time

#id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags
import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags
        
        if (data['lang'] == 'en'):
            iseng = True
        else:
            iseng = False
        
        #case 1: check whether it is a retweet data or not
        if (check_var(data, 'retweeted_status')) :
            
            Created_at = pd.to_datetime(data['retweeted_status']['created_at']).strftime('%B %d, %Y, %r')
            UserID = data['retweeted_status']['user']['id']
            ID = data['retweeted_status']['id']
            retweet_count = data['retweeted_status']['retweet_count']
            favorite_count = data['retweeted_status']['favorite_count']
            reply_count = data['retweeted_status']['reply_count']
            
            #check whether there is extension version of that tweet or not        
            if (check_var(data['retweeted_status'], 'extended_tweet')) :
                Text = data['retweeted_status']['extended_tweet']['full_text']
                for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                    hashtag.append(t['text'])                
            else :
                Text = data['retweeted_status']['text']
                for t in data['retweeted_status']['entities']['hashtags'] :
                    hashtag.append(t['text'])
                    
            #check whether it is retweet of quoted tweet or not
            if (check_var(data['retweeted_status'], 'quoted_status')) :
                
                #check whether there is extension version of the original tweet or not 
                if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                    Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                    for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])               
        
        #case 2: check whether it is a quote tweet data or not
        elif (check_var(data, 'quoted_status')) :
            Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
            UserID = data['user']['id']
            ID = data['id']
            retweet_count = data['retweet_count']
            favorite_count = data['favorite_count']
            reply_count = data['reply_count']
            
            #check whether there is extension version of that quote tweet or not 
            if(check_var(data, 'extended_tweet')) :
                Text = data['extended_tweet']['full_text'] + ' || ' 
            else:
                Text = data['text'] + ' || '
            
            for t in data['entities']['hashtags'] :
                hashtag.append(t['text'])
            
            #check whether there is extension version of the originl tweet or not 
            if (check_var(data['quoted_status'], 'extended_tweet')) :
                Text = Text + data['quoted_status']['extended_tweet']['full_text']
                for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                    hashtag.append(t['text'])                
            else:
                Text = Text + data['quoted_status']['text']
                for t in data['quoted_status']['entities']['hashtags'] :
                    hashtag.append(t['text'])
        
        #if it does not belong to case 1 and case 2, it is an ordinary tweet data
        else :
            Created_at = pd.to_datetime(data['created_at']).strftime('%B %d, %Y, %r')
            UserID = data['user']['id']
            ID = data['id']
            retweet_count = data['retweet_count']
            favorite_count = data['favorite_count']
            reply_count = data['reply_count']
            
            #check whether there is extension version of that tweet or not  
            if(check_var(data, 'extended_tweet')) :
                Text = data['extended_tweet']['full_text']
            else :
                Text = data['text']
                
            for t in data['entities']['hashtags'] :
                hashtag.append(t['text'])
        
        tw = {'id' : ID,
              'user_id' : UserID,
              'created_at' : Created_at,
              'text' : Text,
              'retweet_count' : retweet_count,
              'favorite_count' : favorite_count,
              'reply_count' : reply_count,
              'hashtags': hashtag,
              'isenglish' : iseng
             }
        
        #if the data is already in our database, we update it 
        if (tweet.count_documents({'id' : tw['id']}) > 0) :
            db.tweet.update_one({'id' : tw['id']},
                                {'$set' : {'retweet_count' : tw['retweet_count'],
                                           'favorite_count' : tw['favorite_count'],
                                           'reply_count' : tw['reply_count']}})
        #if it is not, we insert it    
        else :
            tweet.insert_one(tw)
            

tweet.create_index([ ('id', 1) ])

Wall time: 1min 20s


'id_1'

In [18]:
pd.DataFrame(tweet.find()).head(6)

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags,isenglish
0,5ea39aa368c9ac6ee0505721,1249378751349231616,16144221,"April 12, 2020, 04:48:01 PM",wishing death on people is weirdo behavior. ||...,24,50,1,[],True
1,5ea39aa368c9ac6ee0505722,1249397541596286979,1087735689091928064,"April 12, 2020, 06:02:41 PM","In Turkey, there are 300 thousand prisoners an...",23,10,1,[COVID19InTurkeysPrisons],True
2,5ea39aa368c9ac6ee0505723,1249326224964345857,268218622,"April 12, 2020, 01:19:18 PM",Thank You Sir !!\nReally thankful for encourag...,18,24,0,[BSNL],True
3,5ea39aa368c9ac6ee0505724,1249403114614075400,1193535233242664960,"April 12, 2020, 06:24:49 PM","Turkey is so stubborn to change their mind, th...",26,13,0,[COVID19InTurkeysPrisons],True
4,5ea39aa368c9ac6ee0505725,1249316363681910784,14135350,"April 12, 2020, 12:40:06 PM",This image.\nThis quote.\n\n“One of the reason...,3319,6875,230,[],True
5,5ea39aa368c9ac6ee0505726,1249403770360016896,988174833849634816,"April 12, 2020, 06:27:26 PM",SERÁ APENAS COINCIDÊNCIA?\n\nApós escudo do Re...,5,7,0,[],False






# II. User Data Cleaning





In [24]:
## User Table

In [8]:
import pandas as pd
import psycopg2
import json


In [25]:
##Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()
cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

DuplicateDatabase: database "twitter_users" already exists


In [26]:
#Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id varchar(50) PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [27]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [ ]:
#This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count=EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
with open(file, 'r', encoding = 'utf8') as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        
        if (check_var(data, 'retweeted_status')):
            
            user_id = data['retweeted_status']['user']['id_str']
            name = data['retweeted_status']['user']['name']
            user_name = data['retweeted_status']['user']['screen_name']
            verified_status = data['retweeted_status']['user']['verified']
            followers_count = data['retweeted_status']['user']['followers_count']
            friends_count = data['retweeted_status']['user']['friends_count']
            statuses_count = data['retweeted_status']['user']['statuses_count']
            user_location = data['retweeted_status']['user']['location']
            favourites_count= data['retweeted_status']['user']['favourites_count']
            
           
        else :
            user_id = data['user']['id_str']
            name = data['user']['name']
            user_name = data['user']['screen_name']
            verified_status = data['user']['verified']
            followers_count = data['user']['followers_count']
            friends_count = data['user']['friends_count']
            statuses_count = data['user']['statuses_count']
            user_location = data['user']['location']
            favourites_count= data['user']['favourites_count']
            
               
        #insert the data into the PostgreSQL database
        store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count)

In [23]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [27]:
create_pandas_table("SELECT * FROM users_data ").head(6)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,16144221,NUFF,nuffsaidny,False,17112,1515,2599,None,15790
1,268218622,Umesh Agrawal,umesh_agr,False,288,322,258,None,127
2,1001580696077467649,#DerApotheker,ApothekerDer,False,22394,749,16221,None,39417
3,1171484224244744198,Elfpunkt,meElfpunkt,False,3,53,60,None,72
4,3917836273,aniel-ani 🇮🇳,ani_royal007,False,15584,10763,217651,India,50435
5,2950846039,Rizzu,riaz_nov,False,4,63,1107,None,903






# III. Searching Application





In [38]:
## Install PySpark
#!curl -O https://d3kbcqa49mib13.cloudfront.net/spark-2.2.0-bin-hadoop2.7.tgz
#!tar -xvf spark-2.2.0-bin-hadoop2.7.tgz

In [39]:
#Configure your python environment by uncommenting the line below.
#!pip install findspark

In [19]:
## Launch PySpark
import os
import findspark
os.environ["PYSPARK_PYTHON"] = "python3"
findspark.init("spark-2.2.0-bin-hadoop2.7",)

ModuleNotFoundError: No module named 'findspark'

Import components of SparkSQL

In [41]:
from pyspark.sql import SparkSession, Column, Row, functions as F

Initialize the SparkSQL session which contains a basic Spark Context. This may take a few moments to launch the cluster of (typically 4 to 8 python jobs in the background). Note that you are running Spark locally (.master).

In [42]:
spark = (
    SparkSession.builder
        .master("local[*]")
        .appName("SparkLecture694Example")
        .getOrCreate()
)
sc = spark.sparkContext

Implement 1

In [ ]:
# With the Tweet table, a user can get such as the most popular original tweet ranked by the retweets count; 
# or the most popular hashtag ranked by aggregating the lists of hashtags among all tweets we collect

Implement 2

In [ ]:
# With the user table, a user can get such as the most active user ranked by tweets count; 
# or the most active location ranked by tweets count.

In [37]:
# join the tweet table and user table

Implement 3

In [ ]:
# When a user searches for a keyword or hashtag in the Tweet table, the results will show all the original tweets 
# that contain it. We can rank it by number of retweets, number of favorites, number of replies, or sum of them to 
# find the most popular tweet. 

Implement 4

In [ ]:
# We can also link the results with the user table to find out the corresponding users 
# who tweet them and rank the results by the number of followers, number of friends, number of statuses, 
# number of favorites or combination of them.

Implement 5

In [ ]:
# We also save the location information of users in the user table, even though it is self-reported and users 
# can choose whether they want to register or not. We still plan to set the related search in our application. 

# A user can search for the most preferred location ranked by amount of twitter users. 
# When a user searches a hashtag, the results can show the most active location related to the hashtag.